In [1]:
# импорт
import requests, base64, datetime, time, json
from urllib.parse import urlencode

import os
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rrp.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()
from tracks.models import *

In [2]:
# переменные для авторизации Spotify
CLIENT_ID      = 'ddd7b7ba65814c03afa810d062635ae2'
CLIENT_SECRET  = '62b9dd2f26fd47e8ad07e4274fc441f6'
# данные для авторизации Last FM
API_KEY        = '5a0ae1296c8d4388a66c507ba27e92aa'

In [3]:
# подключение к API и прочие сервисные функции, взято на
# "https://github.com/codingforentrepreneurs/30-Days-of-Python/tree/master/tutorial-reference/Day%2019"
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id     = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id     = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in   = data['expires_in'] # seconds
        expires      = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True


In [4]:
# обновить или создать токен
def get_token(client_id, client_secret):
    spotify = SpotifyAPI(client_id, client_secret)
    spotify.perform_auth()
    access_token = spotify.access_token
    return access_token

In [15]:
# список групп для поиска
bands = []
with open('bands.txt', encoding='UTF-8') as f:
    for line in f.readlines():
        bands.append(line.strip())

In [26]:
# заполнение таблицы 'Band'
def populate_bands(bands, client_id, client_secret):
    '''
    Функция проходит по всему списку групп и парсит для
    каждой инфу с сайта Spotify.
    Необходимо передать client_id и client_secret для вызова.
    Заполняет поля:    
    
        band_id         : id группы на Spotify,
        band_name       : название,
        band_genres     : список жанров, определенных для группы,
        band_URL        : ссылка на группу,
        band_followers  : кол-во подписчиков,
        band_popularity : индекс популярности (0-100),
        band_image_300  : ссылка на изображение 300 * 300,
        band_image_64   : ссылка на изображение 64 * 64
    '''
    # обновить или создать токен
    access_token = get_token(client_id, client_secret)

    # поиск групп из списка
    # GET "https://api.spotify.com/v1/search?q=some_name&type=artist"
    for band in bands:
        time.sleep(1)
        # вывод логов
        print(f'Поиcк группы "{band}" ...')

        headers = {"Authorization": f"Bearer {access_token}"}
        endpoint = "https://api.spotify.com/v1/search"
        data = urlencode({"q": f'"{band}"', "type": "artist"})
        lookup_url = f"{endpoint}?{data}"
        request = requests.get(lookup_url, headers=headers).json()

        # обход групп в результате запроса
        artists = request['artists']['items']
        for index, artist in enumerate(artists):
            # сбор всех жанров группы
            genres = ', '.join(request['artists']['items'][index]['genres'])
            # если жанры содержат "russian" - спарсить данные
            if (genres.__contains__ ('russian')) and not \
               (genres.__contains__ ('piano') or genres.__contains__ ('dnb')):
                # вывод логов
                print(f'Найдено совпадение для "{band}" - {artist["name"]}')
                band_dict = { 
                    'band_id'         : artist['id'],
                    'band_name'       : band,
                    'band_genres'     : genres,
                    'band_URL'        : artist['external_urls']['spotify'],
                    'band_followers'  : artist['followers']['total'],
                    'band_popularity' : artist['popularity'],
                    'band_image_300'  : artist['images'][1]['url'],
                    'band_image_64'   : artist['images'][2]['url']
                }
                # первое совпадение - искомая группа
                # создание записи в БД
                Band.objects.create(**band_dict)
                # не запрашивать жанры других групп
                break
    # вывод логов
    print(f'Поиск завершен, найдено групп - {len(Band.objects.all())}')

# populate_bands(bands=bands, client_id=CLIENT_ID, client_secret=CLIENT_SECRET)

In [25]:
# заполнение таблицы 'Album'
def populate_albums(client_id, client_secret):
    '''
    Функция проходит по всему списку альбомов в БД и парсит для
    каждого инфу с сайта Spotify.
    Необходимо передать client_id и client_secret для вызова.
    Заполняет поля:    

        band_id            : id группы на Spotify,    
        band_name          : название группы,
        album_id           : id альбома на Spotify,
        album_name         : название альбома,
        album_type_1       : тип альбома 1,
        album_type_2       : тип альбома 2,
        album_URL          : ссылка на альбом,
        album_release_date : дата выхода,
        album_total_tracks : кол-во треков в альбоме,
        album_image_300    : ссылка на изображение 300 * 300,
        album_image_64     : ссылка на изображение 64 * 64

    '''
    # обновить или создать токен
    access_token = get_token(client_id, client_secret)

    # получение списка альбомов из БД
    bands = Band.objects.all().values_list('band_name', 'band_id', 
                                           'band_image_300', 'band_image_64')
    # создание списка спарсенных альбомов
    albums_ids = []
    # обход альбомов и создание временных переменных
    for band in bands:
        band_name      = band[0]
        band_id        = band[1]
        band_image_300 = band[2]
        band_image_64  = band[3]

        # запрос к Spotify 
        # GET https://api.spotify.com/v1/artists/{id}/albums
        print(f'Поиск альбомов группы "{band_name}" ...')
        # time.sleep(1)
        album_headers    = {"Authorization": f"Bearer {access_token}"}
        album_endpoint   = "https://api.spotify.com/v1/artists/"
        album_lookup_url = f"{album_endpoint}{band_id}/albums"
        album_request    = requests.get(album_lookup_url, headers=album_headers).json()

        # проход в цикле по альбомам в результате запроса
        albums = album_request['items']
        for album in albums:
            #  если нет картинки альбома - взять картинку группы
            try:            
                album_image_300 = album['images'][0]['url']
            except:
                album_image_300 = band_image_300
            try:            
                album_image_64  = album['images'][1]['url']
            except:
                album_image_64  = band_image_64
            # если в дате ошибка - считать год и дописать 01.01
            album_release_date = album['release_date']
            if len(album_release_date) < 10:
                album_release_date = f'{album_release_date[:4]}-01-01'
            # проверка на дублирование альбома
            if album['id'] in albums_ids:
                continue
            else:
                # словарь с данными для создания записи в БД
                album_dict = {
                    'album_id'           : album['id'],
                    'album_name'         : album['name'],
                    'album_type_1'       : album['album_group'],
                    'album_type_2'       : album['album_group'],
                    'album_URL'          : album['external_urls']['spotify'],
                    'album_release_date' : album_release_date,
                    'album_total_tracks' : album['total_tracks'],
                    'album_image_300'    : album_image_300,
                    'album_image_64'     : album_image_64,
                    'band_name'          : band_name,
                    'band_id'            : Band.objects.get(band_id=band_id)
                }
                # внести ID альбома в список спарсенных
                albums_ids.append(album['id'])
                # создание записи в БД
                Album.objects.create(**album_dict)
                # вывод логов
                album_name = album['name']
                print(f'--- Найден альбом "{album_name}"')
    # вывод логов
    print(f'Поиск завершен, найдено альбомов - {len(Album.objects.all())}')
    # удалить список спарсенных альбомов
    del albums_ids

# populate_albums(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)

In [5]:
# удаление треков с ошибками
def del_fail_album(start):
    album_to_del = Album.objects.all().order_by('album_id').values_list('album_id')[start][0]
    Track.objects.filter(album_id=album_to_del).delete()

# заполнение таблицы 'Track'
def populate_tracks(client_id, client_secret, start):
    '''
    Функция проходит по всему списку треков в БД и парсит для
    каждого инфу с сайта Spotify + кол-во прослушиваний с Last FM 
    Необходимо передать client_id и client_secret для вызова.
    В случае сбоя - удалить треки из альбома номер которого выведен в логах
    последним и вызвать функцию с параметром "start" равным номеру этого альбома
    Заполняет поля:

        --- общие свойства ---
        band_id           : id группы на Spotify, 
        band_name         : название группы,
        album_id          : id альбома на Spotify,
        album_name        : название альбома,
        track_id          : id трека на Spotify,
        track_name        : название трека,
        track_disc_number : номер диска в альбоме,
        track_number      : номер трека на диске,
        track_duration    : длительность трека в мс,
        track_image_300   : ссылка на изображение 300 * 300, 
        track_image_64    : ссылка на изображение 64 * 64,
        track_preview     : отрывок из трека 30 секунд,
        track_URL         : ссылка на трек

        --- аудио свойства трека ---
        track_danceability    : 
        track_energy          : 
        track_key             : 
        track_loudness        : 
        track_mode            : 
        track_speechiness     : 
        track_acousticness    : 
        track_instrumentalness: 
        track_liveness        : 
        track_valence         : 
        track_tempo           : 

        --- кол-во прослушиваний с Last FM ---
        track_listeners: прослушано до конца,
        track_playcount: прослушано частично 

    '''
    # обновить или создать токен
    access_token = get_token(client_id, client_secret)

    # получение треков для каждого альбома
    albums = Album.objects.all().values_list('band_name', 'band_id', 
                                             'album_name', 'album_id',
                                             'album_image_300', 'album_image_64') \
                                             .order_by('album_id')

    # проход по всем альбомам в цикле while - 
    # попытка спарсить все треки и всею инфу по каждому треку
    # если ошибка - удаление всех треков альбома и повтор
    start  = start
    finish = len(albums)

    while start < finish:
        try:
            album = albums[start]
            # создание временных переменных
            band_name       = album[0]  # 'Ленинград'
            band_id         = album[1]  # '2a3EJeQjcAelthFGr5bzQC' 
            album_name      = album[2]  # 'Пуля'
            album_id        = album[3]  # '00QHUtm8MHpeoYHiqXRhX6'
            album_image_300 = album[4]  # 'https://i.scdn.co/image/ab.....'
            album_image_64  = album[5]  # 'https://i.scdn.co/image/ab.....'

            # запрос к Spotify
            #  GET https://api.spotify.com/v1/albums/{id}/tracks
            time.sleep(1)  
            track_headers = {"Authorization": f"Bearer {access_token}"}
            track_endpoint = "https://api.spotify.com/v1/albums/"
            track_lookup_url = f"{track_endpoint}{album_id}/tracks"
            track_request = requests.get(track_lookup_url, headers=track_headers).json()
            # вывод логов
            print(f'Поиск треков из {start} альбома "{band_name}" - "{album_name}"')

            # обход треков в результате запроса
            tracks_by_album = track_request['items']
            for track in tracks_by_album:
                
                track_name = track['name']
                track_id   = track['id']
                # если нет ссылки на превью - взять дефолт
                if track['preview_url']:
                    track_preview = track['preview_url']
                # либо продолжить по плану   
                else:
                    # логи
                    print(f'--- !!! Preview трека "{band_name}" - "{track_name}" не найдено')
                    track_preview = Track.objects.all().values_list('track_preview')[0][0]
                # вывод логов
                print(f'--- Найден трек  "{track_name}"')
                # словарь с общими данными трека
                track_info = {
                    'track_id'          : track['id'],
                    'track_name'        : track['name'],
                    'track_disc_number' : track['disc_number'],
                    'track_number'      : track['track_number'],
                    'track_duration'    : track['duration_ms'],
                    'album_id'          : Album.objects.get(album_id=album_id),
                    'album_name'        : album_name,
                    'band_id'           : Band.objects.get(band_id=band_id), 
                    'band_name'         : band_name,
                    'track_image_300'   : album_image_300, 
                    'track_image_64'    : album_image_64,
                    'track_preview'     : track_preview,
                    'track_URL'         : track['external_urls']['spotify']
                }
                
                # получение аудио фичей из Spotify
                # GET https://api.spotify.com/v1/audio-features/{id}
                time.sleep(1)
                # логи
                print(f'--- Поиск аудио фичей для трека "{band_name}" - "{track_name}"')
                
                feautures_headers    = {"Authorization": f"Bearer {access_token}"}
                feautures_endpoint   = "https://api.spotify.com/v1/audio-features/"
                feautures_lookup_url = f"{feautures_endpoint}{track_id}"
                feautures_request    = requests.get(feautures_lookup_url, headers=feautures_headers).json()

                # если нет записей - пропустить (в БД для всех фичей дефолт = 0.)
                try:       
                    track_audio_features = {
                        'track_danceability'    : feautures_request['danceability'],
                        'track_energy'          : feautures_request['energy'],
                        'track_key'             : feautures_request['key'],
                        'track_loudness'        : feautures_request['loudness'],
                        'track_mode'            : feautures_request['mode'],
                        'track_speechiness'     : feautures_request['speechiness'],
                        'track_acousticness'    : feautures_request['acousticness'],
                        'track_instrumentalness': feautures_request['instrumentalness'],
                        'track_liveness'        : feautures_request['liveness'],
                        'track_valence'         : feautures_request['valence'],
                        'track_tempo'           : feautures_request['tempo'],
                    }
                    track_info.update(track_audio_features)
                    # логи
                    print(f'--- Аудио фичи для трека "{band_name}" - "{track_name}" найдены')
                except:
                    # логи
                    print(f'--- !!! Аудио фичи для трека "{band_name}" - "{track_name}" не найдены')


                # получение данных о прослушивании трека на Last FM
                # GET http://ws.audioscrobbler.com/2.0/?method=track.getInfo&api_key={API_KEY}
                # &artist={ARTIST_NAME}&track={TRACK_NAME}&format=json
                # вывод логов
                print(f'--- Поиск информации о прослушиваниях "{band_name}" - "{track_name}"')
                time.sleep(5)
                try:
                    lastFM_endpoint   = "http://ws.audioscrobbler.com/2.0/?method=track.getInfo&api_key="
                    lastFM_lookup_url = f"{lastFM_endpoint}{API_KEY}&artist={band_name}&track={track_name}&format=json"
                    lastFM_request   = requests.get(lastFM_lookup_url).json()

                    last_fm_features = {
                        'track_listeners': lastFM_request['track']['listeners'],
                        'track_playcount': lastFM_request['track']['playcount'] 
                    }

                    track_info.update(last_fm_features)
                    # вывод логов
                    print(f'--- Информация о прослушиваниях "{band_name}" - "{track_name}" добавлена')
                except:
                    # вывод логов
                    print(f'--- !!! Информация о прослушиваниях "{band_name}" - "{track_name}" не найдена')
                    pass

                # создание записи в БД
                Track.objects.create(**track_info)
                # вывод логов
                print(f'--- Добавлен трек  "{track_name}"')

            start += 1

        except:
            # вывод логов
            print(f'!!! Повторная загрузка информации об альбоме "{band_name}" - "{album_name}"')
            # обновить или создать токен
            access_token = get_token(client_id, client_secret)
            # в случае ошибки - удалить все трекит текущего альбома
            # и начать парсинг этого альбома сначала
            Track.objects.filter(album_id=album_id).delete()

    # финальные логи
    print(f'Добавлено {len(Track.objects.all())} треков')

# при ошибке парсинга установить значение "start" из логов
# при первом запуске не менять  
START = 0
del_fail_album(START)

# populate_tracks(client_id=CLIENT_ID, client_secret=CLIENT_SECRET, start=START)

Поиск треков из 2060 альбома "Полюса" - "Точка невозврата (Remastered)"
--- Найден трек  "Кино - Remastered"
--- Поиск аудио фичей для трека "Полюса" - "Кино - Remastered"
--- Аудио фичи для трека "Полюса" - "Кино - Remastered" найдены
--- Поиск информации о прослушиваниях "Полюса" - "Кино - Remastered"
--- Информация о прослушиваниях "Полюса" - "Кино - Remastered" добавлена
--- Добавлен трек  "Кино - Remastered"
--- Найден трек  "Витки - Remastered"
--- Поиск аудио фичей для трека "Полюса" - "Витки - Remastered"
--- Аудио фичи для трека "Полюса" - "Витки - Remastered" найдены
--- Поиск информации о прослушиваниях "Полюса" - "Витки - Remastered"
--- Информация о прослушиваниях "Полюса" - "Витки - Remastered" добавлена
--- Добавлен трек  "Витки - Remastered"
--- Найден трек  "Делай меня живым - Remastered"
--- Поиск аудио фичей для трека "Полюса" - "Делай меня живым - Remastered"
--- Аудио фичи для трека "Полюса" - "Делай меня живым - Remastered" найдены
--- Поиск информации о прослушив

In [32]:
# Track.objects.filter(album_id=start).delete()

(6, {'tracks.Track': 6})

In [25]:
# импорт
from slugify import slugify
from bs4 import BeautifulSoup as bs

In [ ]:

track_list = Track.objects.all().values_list('band_id', 'band_name', 
                                            'album_id', 'album_name',
                                            'track_id', 'track_name') \
                                            .order_by('track_id')[:20]
for track in track_list:
    band_id    = track[0]   # '2GUwb2rxMKePzxDi94EEoZ'
    band_name  = track[1]   # 'Ночные снайперы'
    album_id   = track[2]   # '3z9HvGReqatzDLy2DL1w18'
    album_name = track[3]   # 'Koshika'
    track_id   = track[4]   # '0032CHOcRVH98tHdVYFXRN'
    track_name = track[5]   # 'катастрофически'

    print(band_name, track_name)

    text_dict = { 
        'band_id'    : band_id,
        'band_name'  : band_name,
        'album_id'   : album_id,
        'album_name' : album_name,
        'track_id'   : Track.objects.get(track_id=track_id),
        'track_name' : track_name
    }

    vse_pesni_req  = requests.get(f'https://vse-pesni.com/song/{slugify(band_name)}-{slugify(track_name)}/')
    lyricshare_req = requests.get(f'http://www.lyricshare.net/ru/{slugify(band_name)}/{slugify(track_name)}.html')
    megalyrics_req = requests.get(f'http://www.megalyrics.ru/lyric/{slugify(band_name)}/{slugify(track_name)}.htm')

    if vse_pesni_req.ok:
        vse_pesni_html = bs(vse_pesni_req.text, 'lxml')

        song_text = vse_pesni_html.find('div', id='page') \
                                .find('div', id='main') \
                                .find('div', id='content') \
                                .find('div', class_='entry-content') \
                                .find('div', class_='song_text')

        song_text = (str(song_text.__dict__['contents'][0]) + \
                    str(song_text.__dict__['contents'][2])) \
                    .replace('\n', '')\
                    .replace('\t', '')\
                    .replace('\r', ' ')\
                    .strip()

        song_text_dict = {'song_text': song_text}
        text_dict.update(song_text_dict)
        print(f'Загружен текст с сайта "vse_pesni"')    



    # if vse_pesni_req.ok:
    #     try:
    #         vse_pesni_html = bs(vse_pesni_req.text, 'lxml')

    #         song_text = vse_pesni_html.find('div', id='page') \
    #                                 .find('div', id='main') \
    #                                 .find('div', id='content') \
    #                                 .find('div', class_='entry-content') \
    #                                 .find('div', class_='song_text')
                                    
    #         song_text = str(song_text.__dict__['contents'][0] + \
    #                         song_text.__dict__['contents'][2]) \
    #                         .replace('\n', '') \
    #                         .replace('\t', '') \
    #                         .replace('\r', ' ').strip()

    #         print(song_text)

    #         song_text_dict = {'song_text': song_text}
    #         text_dict.update(**song_text_dict)
    #         print(f'Загружен текст с сайта "vse_pesni"')
    #     except:
    #         pass
            
    # elif lyricshare_req.ok:
    #     try:
    #         lyricshare_req_html = bs(vse_pesni_req.text, 'lxml')

    #         song_text = lyricshare_req_html.find('div', id='page') \
    #                                 .find('div', id='main') \
    #                                 .find('div', id='content') \
    #                                 .find('div', class_='entry-content') \
    #                                 .find('div', class_='song_text')

    #         song_text = song_text.text.replace('ПРИПЕВ', '') \
    #                                 .replace('\n', ' ').strip()

    #         song_text_dict = {'song_text': song_text}
    #         text_dict.update(song_text_dict)
    #         print(f'Загружен текст с сайта "lyricshare"')
    #     except:
    #         pass

    # elif megalyrics_req.ok:
    #     try:
    #         megalyrics_html = bs(megalyrics_req.text, 'lxml')
            
    #         song_text = megalyrics_html.find('div', id='wrap') \
    #                                 .find('div', id='page') \
    #                                 .find('div', id='page_main') \
    #                                 .find('div', id='songs_show') \
    #                                 .find('div', class_='wrap') \
    #                                 .find('div', id='txt') \
    #                                 .find('div', class_='text separate') \
    #                                 .find('div', class_='text_inner')

    #         song_text = str(song_text).replace('<br/>', ' ') \
    #                                 .replace('<div class="text_inner">', '') \
    #                                 .replace('<div id="native_roll">', '') \
    #                                 .replace('</div>', '') \
    #                                 .strip()

    #         song_text_dict = {'song_text': song_text}
    #         text_dict.update(song_text_dict)
    #         print(f'Загружен текст с сайта "megalyrics"')
    #     except:
    #         pass

    # создание записи в БД
    Text.objects.create(**text_dict)


In [ ]:
vse_pesni_req  = requests.get(f'https://vse-pesni.com/song/{band_name}-{track_name}/')
lyricshare_req = requests.get(f'http://www.lyricshare.net/ru/{band_name}/{track_name}.html')
megalyrics_req = requests.get(f'http://www.megalyrics.ru/lyric/{band_name}/{track_name}.htm')

In [ ]:
print(vse_pesni_req.ok, lyricshare_req.ok, megalyrics_req.ok) 


In [ ]:
vse_pesni_html = bs(vse_pesni_req.text, 'lxml')

song_text = vse_pesni_html.find('div', id='page') \
                          .find('div', id='main') \
                          .find('div', id='content') \
                          .find('div', class_='entry-content') \
                          .find('div', class_='song_text')

In [ ]:
(song_text.__dict__['contents'][0] + song_text.__dict__['contents'][2]).replace('\n', '').replace('\t', '').replace('\r', ' ').strip()

In [ ]:
lyricshare_html = bs(lyricshare_req.text, 'lxml')

song_text = lyricshare_html.find('div', id='wrapper') \
                           .find('div', class_='wrapper-inner') \
                           .find('div', id='colone-container') \
                           .find('div', class_='col-one') \
                           .find('div', id='textPesniDiv') \
                           .find('div', id='textPesniDiv-outer') \
                           .find('p', id='textpesni')

In [ ]:
song_text.text.replace('ПРИПЕВ', '').replace('\n', ' ').strip()

In [15]:
track_list = Track.objects.all().values_list('band_id', 'band_name', 
                                            'album_id', 'album_name',
                                            'track_id', 'track_name') \
                                            .order_by('track_id')[:5]

In [18]:
track_list[3]

('2GUwb2rxMKePzxDi94EEoZ',
 'Ночные снайперы',
 '3z9HvGReqatzDLy2DL1w18',
 'Koshika',
 '0032CHOcRVH98tHdVYFXRN',
 'катастрофически')

In [ ]:
megalyrics_html = bs(megalyrics_req.text, 'lxml')
song_text = megalyrics_html.find('div', id='wrap') \
                           .find('div', id='page') \
                           .find('div', id='page_main') \
                           .find('div', id='songs_show') \
                           .find('div', class_='wrap') \
                           .find('div', id='txt') \
                           .find('div', class_='text separate') \
                           .find('div', class_='text_inner')
print(str(song_text).replace('<br/>', ' ').replace('<div class="text_inner">', '').replace('<div id="native_roll">', '').replace('</div>', '').strip())